In [7]:
from interface import ObjectInterface
from ddl import Interpreter
import utils
import logging
import os
import examples.randomdata as rd
from decimal import Decimal
import random

FILENAME_DATABASE = 'data/database.db'
FILENAME_STRUCTURE = 'setup/example_structure.ddl'
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.DEBUG)

def get_interface() -> ObjectInterface:
    interface = ObjectInterface(FILENAME_DATABASE)
    interface.connect()
    return interface

### Datenbank neu aufsetzen und Datenmodell einspielen

In [8]:
# Datenbank löschen (wenn existiert)
if os.path.exists(FILENAME_DATABASE):
    os.remove(FILENAME_DATABASE)

# Datenmodell einspielen
with get_interface() as interface:
    interface.setup()
    interpreter = Interpreter(interface)
    with open(FILENAME_STRUCTURE, 'r') as file:
        interpreter.run(file.read())
    interface.commit()

[2024-01-25 17:21:33] DEBUG: Setup successful
[2024-01-25 17:21:33] DEBUG: Created datatype void (TINYINT, no read transformer, no write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype int (INTEGER, no read transformer, no write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype shorttext (VARCHAR(64), no read transformer, no write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype longtext (VARCHAR(256), no read transformer, no write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype text (TEXT, no read transformer, no write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype date (DATE, read transformer, write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype datetime (DATETIME, read transformer, write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype array (BLOB, read transformer, write transformer)
[2024-01-25 17:21:33] DEBUG: Created datatype zipcode (VARCHAR(5), no read transformer, no write transformer)
[2024-01-25 17:2

[2024-01-25 17:21:33] DEBUG: Assigned birthday to Person
[2024-01-25 17:21:33] DEBUG: Created new class Customer as subclass of Person
[2024-01-25 17:21:33] DEBUG: Assigned street to Customer
[2024-01-25 17:21:33] DEBUG: Assigned house_number to Customer
[2024-01-25 17:21:33] DEBUG: Assigned zip to Customer
[2024-01-25 17:21:33] DEBUG: Assigned city to Customer
[2024-01-25 17:21:33] DEBUG: Created new class Product
[2024-01-25 17:21:33] DEBUG: Assigned name to Product
[2024-01-25 17:21:33] DEBUG: Assigned price to Product
[2024-01-25 17:21:33] DEBUG: Created new class OrderPosition
[2024-01-25 17:21:33] DEBUG: Assigned amount to OrderPosition
[2024-01-25 17:21:33] DEBUG: Created new reference position_to_product between class OrderPosition and Product
[2024-01-25 17:21:33] DEBUG: Created new class Order
[2024-01-25 17:21:33] DEBUG: Assigned creation_time to Order
[2024-01-25 17:21:33] DEBUG: Created new reference order_to_customer between class Order and Customer
[2024-01-25 17:21:33] 

### Kunden einfügen

In [9]:
rd.set_seed(2024)

def get_random_customer():
    customer = rd.get_random_person()
    customer.update(rd.get_random_address())
    return customer

print(get_random_customer())

{'first_name': 'Ottilie', 'last_name': 'Koch', 'birthday': datetime.date(2008, 9, 18), 'street': 'Schröderstraße', 'house_number': '455c', 'zip': '44763', 'city': 'Fuchsdorf'}


In [10]:
n_customers = 100
customer_ids = []
with get_interface() as interface:
    c_customer = interface.get_class(name='Customer')
    for i in range(n_customers):
        customer_ids.append(interface.create_object(c_customer, **get_random_customer()).id)
print(f"Inserted {n_customers} customers")

Inserted 100 customers


### Produkte einfügen

In [11]:
product_ids = []
products = [
    ['Rennrad', Decimal('995.99')],
    ['BMX', Decimal('195.99')],
    ['Hollandrad', Decimal('249.99')],
    ['Tourenrad', Decimal('549.99')],
    ['Kette', Decimal('19.99')],
    ['Klingel', Decimal('5.99')],
    ['Schlauch 28 Zoll', Decimal('7.99')],
    ['Schlauch 26 Zoll', Decimal('7.49')],
    ['Reifen 28 Zoll', Decimal('18.99')],
    ['Reifen 26 Zoll', Decimal('17.99')],
    ['Kettenfett', Decimal('3.99')],
    ['Helm', Decimal('144.99')]
]
with get_interface() as interface:
    c_product = interface.get_class(name='Product')
    for p in products:
        product_ids.append(interface.create_object(c_product, name=p[0], price=p[1]).id)
print(f"Inserted {len(products)} products")

Inserted 12 products


### Bestellungen einfügen

In [14]:
n_orders = 40
with get_interface() as interface:
    c_order = interface.get_class(name='Order')
    c_position = interface.get_class(name='OrderPosition')
    ref_customer = interface.get_reference(name='order_to_customer')
    ref_positions = interface.get_reference(name='order_to_positions')
    ref_product = interface.get_reference(name='position_to_product')
    for i in range(n_orders):
        n_products = random.randint(1, 10)

        # Bestellung erstellen und Kunde verknüpfen
        customer = interface.get_object(random.choice(customer_ids))
        order = interface.create_object(c_order)
        order.bind(ref_customer, [customer])

        # Positionen verknüpfen
        positions = []
        for k in range(n_products):
            product = interface.get_object(random.choice(product_ids))
            position = interface.create_object(c_position, amount=random.randint(1, 10))
            position.bind(ref_product, [product])
            positions.append(position)
        order.bind(ref_positions, positions)

        print(f"{customer['full_name']} bought {n_products} products")

AttributeError: 'NoneType' object has no attribute 'id'